In [2]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings

warnings.filterwarnings("ignore")

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
from manual_ensemble import *

# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe, Trials
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [3]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = [
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
]

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges'] = (
    tcc['TotalCharges'].apply(lambda x: x if x != ' ' else np.nan).astype(float)
)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No', value=0, inplace=True)

In [4]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

In [5]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month'] - 1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month'] - 1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(
    enc.transform(X_train_seq).toarray(), columns=cate_colName(enc, seq_new, drop=None)
)

X_test_seq = pd.DataFrame(
    enc.transform(X_test_seq).toarray(), columns=cate_colName(enc, seq_new, drop=None)
)

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [5]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(
    ord_enc.transform(X_train[category_cols]), columns=category_cols
)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(
    ord_enc.transform(X_test[category_cols]), columns=category_cols
)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

In [6]:
# 本节新增第三方库
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [7]:
class VotingClassifier_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, estimators, voting="hard", weights=None, thr=0.5):
        self.estimators = estimators
        self.voting = voting
        self.weights = weights
        self.thr = thr

    def fit(self, X, y):
        VC = VotingClassifier(
            estimators=self.estimators, voting=self.voting, weights=self.weights
        )

        VC.fit(X, y)
        self.clf = VC

        return self

    def predict_proba(self, X):
        if self.voting == "soft":
            res_proba = self.clf.predict_proba(X)
        else:
            res_proba = None
        return res_proba

    def predict(self, X):
        if self.voting == "soft":
            res = (self.clf.predict_proba(X)[:, 1] >= self.thr) * 1
        else:
            res = self.clf.predict(X)
        return res

    def score(self, X, y):
        acc = accuracy_score(self.predict(X), y)
        return acc

In [8]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)

# 训练集特征
X_train1 = X_train_OE.loc[train_part_index_l[0]]
X_train2 = X_train_OE.loc[train_part_index_l[1]]
X_train3 = X_train_OE.loc[train_part_index_l[2]]
X_train4 = X_train_OE.loc[train_part_index_l[3]]
X_train5 = X_train_OE.loc[train_part_index_l[4]]

# 验证集特征
X_eval1 = X_train_OE.loc[eval_index_l[0]]
X_eval2 = X_train_OE.loc[eval_index_l[1]]
X_eval3 = X_train_OE.loc[eval_index_l[2]]
X_eval4 = X_train_OE.loc[eval_index_l[3]]
X_eval5 = X_train_OE.loc[eval_index_l[4]]

# 训练集标签
y_train1 = y_train.loc[train_part_index_l[0]]
y_train2 = y_train.loc[train_part_index_l[1]]
y_train3 = y_train.loc[train_part_index_l[2]]
y_train4 = y_train.loc[train_part_index_l[3]]
y_train5 = y_train.loc[train_part_index_l[4]]

# 验证集标签
y_eval1 = y_train.loc[eval_index_l[0]]
y_eval2 = y_train.loc[eval_index_l[1]]
y_eval3 = y_train.loc[eval_index_l[2]]
y_eval4 = y_train.loc[eval_index_l[3]]
y_eval5 = y_train.loc[eval_index_l[4]]

train_set = [
    (X_train1, y_train1),
    (X_train2, y_train2),
    (X_train3, y_train3),
    (X_train4, y_train4),
    (X_train5, y_train5),
]

eval_set = [
    (X_eval1, y_eval1),
    (X_eval2, y_eval2),
    (X_eval3, y_eval3),
    (X_eval4, y_eval4),
    (X_eval5, y_eval5),
]

In [9]:
# 随机森林模型组
grid_RF_1 = load("./model/grid_RF_1.joblib")
grid_RF_2 = load("./model/grid_RF_2.joblib")
grid_RF_3 = load("./model/grid_RF_3.joblib")
grid_RF_4 = load("./model/grid_RF_4.joblib")
grid_RF_5 = load("./model/grid_RF_5.joblib")

RF_1 = grid_RF_1.best_estimator_
RF_2 = grid_RF_2.best_estimator_
RF_3 = grid_RF_3.best_estimator_
RF_4 = grid_RF_4.best_estimator_
RF_5 = grid_RF_5.best_estimator_

RF_l = [RF_1, RF_2, RF_3, RF_4, RF_5]

# 决策树模型组
grid_tree_1 = load("./model/grid_tree_1.joblib")
grid_tree_2 = load("./model/grid_tree_2.joblib")
grid_tree_3 = load("./model/grid_tree_3.joblib")
grid_tree_4 = load("./model/grid_tree_4.joblib")
grid_tree_5 = load("./model/grid_tree_5.joblib")

tree_1 = grid_tree_1.best_estimator_
tree_2 = grid_tree_2.best_estimator_
tree_3 = grid_tree_3.best_estimator_
tree_4 = grid_tree_4.best_estimator_
tree_5 = grid_tree_5.best_estimator_

tree_l = [tree_1, tree_2, tree_3, tree_4, tree_5]

# 逻辑回归模型组
grid_lr_1 = load("./model/grid_lr_1.joblib")
grid_lr_2 = load("./model/grid_lr_2.joblib")
grid_lr_3 = load("./model/grid_lr_3.joblib")
grid_lr_4 = load("./model/grid_lr_4.joblib")
grid_lr_5 = load("./model/grid_lr_5.joblib")

lr_1 = grid_lr_1.best_estimator_
lr_2 = grid_lr_2.best_estimator_
lr_3 = grid_lr_3.best_estimator_
lr_4 = grid_lr_4.best_estimator_
lr_5 = grid_lr_5.best_estimator_

lr_l = [lr_1, lr_2, lr_3, lr_4, lr_5]

In [10]:
eval1_predict_proba_RF = pd.Series(
    RF_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index
)
eval2_predict_proba_RF = pd.Series(
    RF_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index
)
eval3_predict_proba_RF = pd.Series(
    RF_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index
)
eval4_predict_proba_RF = pd.Series(
    RF_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index
)
eval5_predict_proba_RF = pd.Series(
    RF_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index
)

eval_predict_proba_RF = pd.concat(
    [
        eval1_predict_proba_RF,
        eval2_predict_proba_RF,
        eval3_predict_proba_RF,
        eval4_predict_proba_RF,
        eval5_predict_proba_RF,
    ]
).sort_index()

eval1_predict_proba_tree = pd.Series(
    tree_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index
)
eval2_predict_proba_tree = pd.Series(
    tree_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index
)
eval3_predict_proba_tree = pd.Series(
    tree_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index
)
eval4_predict_proba_tree = pd.Series(
    tree_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index
)
eval5_predict_proba_tree = pd.Series(
    tree_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index
)

eval_predict_proba_tree = pd.concat(
    [
        eval1_predict_proba_tree,
        eval2_predict_proba_tree,
        eval3_predict_proba_tree,
        eval4_predict_proba_tree,
        eval5_predict_proba_tree,
    ]
).sort_index()

eval1_predict_proba_lr = pd.Series(
    lr_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index
)
eval2_predict_proba_lr = pd.Series(
    lr_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index
)
eval3_predict_proba_lr = pd.Series(
    lr_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index
)
eval4_predict_proba_lr = pd.Series(
    lr_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index
)
eval5_predict_proba_lr = pd.Series(
    lr_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index
)

eval_predict_proba_lr = pd.concat(
    [
        eval1_predict_proba_lr,
        eval2_predict_proba_lr,
        eval3_predict_proba_lr,
        eval4_predict_proba_lr,
        eval5_predict_proba_lr,
    ]
).sort_index()

In [11]:
test_predict_proba_RF = []

for i in range(5):
    test_predict_proba_RF.append(RF_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_RF = np.array(test_predict_proba_RF)
test_predict_proba_RF = test_predict_proba_RF.mean(0)

test_predict_proba_tree = []

for i in range(5):
    test_predict_proba_tree.append(tree_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_tree = np.array(test_predict_proba_tree)
test_predict_proba_tree = test_predict_proba_tree.mean(0)

test_predict_proba_lr = []

for i in range(5):
    test_predict_proba_lr.append(lr_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_lr = np.array(test_predict_proba_lr)
test_predict_proba_lr = test_predict_proba_lr.mean(0)

In [17]:
from sklearn.datasets import fetch_california_housing

In [18]:
cal_housing = fetch_california_housing()

X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)

y = cal_housing.target

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=12
)

In [26]:
X_train.shape

(16512, 8)

In [27]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [28]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,6.6134,4.0,6.560729,0.939271,1552.0,3.141700,37.93,-121.97
1,2.3578,41.0,5.455598,1.007722,1070.0,4.131274,32.80,-117.15
2,5.5111,16.0,5.716747,1.037905,3903.0,2.689869,34.25,-118.61
3,8.1124,52.0,6.623188,1.019324,1153.0,2.785024,34.11,-118.14
4,6.2957,25.0,6.627832,1.008091,2128.0,3.443366,37.25,-121.81


In [29]:
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
)
from sklearn.metrics import mean_squared_error

In [30]:
# 随机森林
reg1 = RandomForestRegressor(random_state=12).fit(X_train, y_train)
print("The results of RandomForest:")
print(
    "Train-MSE: %f, Test-MSE: %f"
    % (
        mean_squared_error(reg1.predict(X_train), y_train),
        mean_squared_error(reg1.predict(X_test), y_test),
    )
)

# 极端随机树
reg2 = ExtraTreesRegressor(random_state=12).fit(X_train, y_train)
print("The results of ExtraTrees:")
print(
    "Train-MSE: %f, Test-MSE: %f"
    % (
        mean_squared_error(reg2.predict(X_train), y_train),
        mean_squared_error(reg2.predict(X_test), y_test),
    )
)

# GBDT
reg3 = GradientBoostingRegressor(random_state=12).fit(X_train, y_train)
print("The results of GBDT:")
print(
    "Train-MSE: %f, Test-MSE: %f"
    % (
        mean_squared_error(reg3.predict(X_train), y_train),
        mean_squared_error(reg3.predict(X_test), y_test),
    )
)

The results of RandomForest:
Train-MSE: 0.035168, Test-MSE: 0.250861
The results of ExtraTrees:
Train-MSE: 0.000000, Test-MSE: 0.243643
The results of GBDT:
Train-MSE: 0.260555, Test-MSE: 0.277794


In [31]:
# 随机森林
reg1 = RandomForestRegressor(random_state=12)
print("The results of RandomForest:")
print(
    "Train-cross-mean: %f"
    % -(
        cross_val_score(
            reg1, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        )
    ).mean()
)

# 极端随机树
reg2 = ExtraTreesRegressor(random_state=12)
print("The results of ExtraTrees:")
print(
    "Train-cross-mean: %f"
    % -(
        cross_val_score(
            reg2, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        )
    ).mean()
)

# GBDT
reg3 = GradientBoostingRegressor(random_state=12)
print("The results of GBDT:")
print(
    "Train-cross-mean: %f"
    % -(
        cross_val_score(
            reg3, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        )
    ).mean()
)

The results of RandomForest:
Train-cross-mean: 0.259973
The results of ExtraTrees:
Train-cross-mean: 0.258106
The results of GBDT:
Train-cross-mean: 0.287060


In [32]:
RF_space = {
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 20, 1),
    "min_samples_split": hp.quniform("min_samples_split", 2, 20, 1),
    "max_depth": hp.quniform("max_depth", 2, 50, 1),
    "max_leaf_nodes": hp.quniform("max_leaf_nodes", 50, 300, 1),
    "n_estimators": hp.quniform("n_estimators", 50, 500, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 0.8),
}

In [33]:
def RF_param_objective(params, train=True):
    # 超参数读取
    min_samples_leaf = int(params["min_samples_leaf"])
    min_samples_split = int(params["min_samples_split"])
    max_depth = int(params["max_depth"])
    max_leaf_nodes = int(params["max_leaf_nodes"])
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    # 模型创建
    reg_RF = RandomForestRegressor(
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        n_estimators=n_estimators,
        max_samples=max_samples,
    )

    if train == True:
        res = -cross_val_score(
            reg_RF, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=15
        ).mean()
    else:
        res = reg_RF.fit(X_train, y_train)

    return res

In [34]:
def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )

In [35]:
RF_best_param = RF_param_search()

100%|██████████| 500/500 [1:04:14<00:00,  7.71s/trial, best loss: 0.28465633940450197]


In [36]:
RF_best_param

{'max_depth': 31.0,
 'max_leaf_nodes': 299.0,
 'max_samples': 0.7718695504904937,
 'min_samples_leaf': 3.0,
 'min_samples_split': 20.0,
 'n_estimators': 307.0}

In [37]:
RF_reg = RF_param_objective(RF_best_param, train=False)
RF_reg

RandomForestRegressor(max_depth=31, max_leaf_nodes=299,
                      max_samples=0.7718695504904937, min_samples_leaf=3,
                      min_samples_split=20, n_estimators=307)

In [38]:
mean_squared_error(RF_reg.predict(X_train), y_train), mean_squared_error(
    RF_reg.predict(X_test), y_test
)

(0.19617896082126313, 0.2761455219853933)

#### 大范围超参数搜索优化（方案A）

&emsp;&emsp;首先也是最容易想到的策略就是增加搜索的超参数，以此降低模型过拟合风险，并且同时增加各超参数搜索范围，以此提升模型泛化能力。这当然是理论上的最优方案，但实际执行难度较大，最核心的困难就在于算力不够。目前的数据集在当前搜索空间范围内迭代500次需要1小时时间，而如果增加超参数数量、并增加搜索空间，超参数优化计算所需时间将呈指数级上升。此外，对于部分模型的部分超参数甚至根本无法带入进行搜索，例如回归随机森林中的criterion参数，根据参数说明可知，当criterion取值为absolute_error时将极大程度影响计算效率、计算时间将大幅提升，这里我们可以简单进行测试：

In [39]:
start = time.time()
RandomForestRegressor().fit(X_train, y_train)
print(time.time() - start)

5.7389304637908936


In [40]:
start = time.time()
RandomForestRegressor(criterion="absolute_error").fit(X_train, y_train)
print(time.time() - start)

283.6328864097595


能够发现计算时间增幅达50倍！当然这不仅仅是回归随机森林单独模型的问题，很多模型在处理回归问题时都有类似的超参数设置情况，稍后建模的回归GBDT模型更是有过之而无不及。

In [41]:
# 重新设计超参数搜索
max_features_range = ["auto", "sqrt", "log2", None] + np.arange(0.1, 1., 0.1).tolist()
max_features_range

['auto',
 'sqrt',
 'log2',
 None,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9]

In [42]:
RF_space = {
    "max_features": hp.choice("max_features", max_features_range),
    "n_estimators": hp.quniform("n_estimators", 200, 800, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 1),
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 50, 1),
    "min_samples_split": hp.quniform("min_samples_split", 2, 50, 1),
    "max_depth": hp.quniform("max_depth", 20, 120, 1),
    "max_leaf_nodes": hp.quniform("max_leaf_nodes", 200, 600, 1),
}

In [43]:
def RF_param_objective(params, train=True):
    # 超参数读取
    min_samples_leaf = int(params["min_samples_leaf"])
    min_samples_split = int(params["min_samples_split"])
    max_depth = int(params["max_depth"])
    max_leaf_nodes = int(params["max_leaf_nodes"])
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_RF = RandomForestRegressor(
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        random_state=12,
    )

    if train == True:
        res = -cross_val_score(
            reg_RF, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        ).mean()
    else:
        res = reg_RF.fit(X_train, y_train)

    return res

In [44]:
def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )

In [45]:
RF_best_param = RF_param_search(1000)

100%|██████████| 1000/1000 [2:14:50<00:00,  8.09s/trial, best loss: 0.2572447927363112]  


In [46]:
RF_reg_A = RF_param_objective(RF_best_param, train=False)

In [47]:
mean_squared_error(RF_reg_A.predict(X_train), y_train), mean_squared_error(
    RF_reg_A.predict(X_test), y_test
)

(0.13417130464725555, 0.25010856874888276)

#### 更高效的搜索流程（方案B）

&emsp;&emsp;这里我们考虑另一种更高效的搜索流程：直接舍弃基础学习器的超参数搜索，只搜索集成相关超参数。这里需要注意的是，舍弃基础学习器的超参数搜索其实就等于带入基础学习器的默认超参数组，而基础学习器的默认超参数组是不会剪枝的、即默认执行最高模型复杂度的模型训练，每个基础学习器实际上是容易过拟合的，但是考虑到回归问题本身就要求更复杂的模型进行训练，且已被上述搜索过程验证，因此基础学习器选取默认超参数并不会对预测结果有较大的影响。并且除了基础学习器的超参数外、对模型泛化能力影响更大的继承超参数仍然是朝着泛化能力更强的方向进行搜索，外加舍弃了这部分超参数搜索，整体超参数搜索的速度将变得更快，这将有助于在更短的时间内快速搜索出一组更好的结果。        
&emsp;&emsp;但是需要注意，如果接下来采用方案二进行超参数搜索，当我们舍弃了更多的基础学习器的超参数的时候，最终训练集的预测结果将会有更大的过拟合风险，因此在方案二的超参数搜索时，仍然需要坚持使用交叉验证结果作为模型泛化能力的评分标准。

In [48]:
RF_space = {
    "max_features": hp.choice("max_features", max_features_range),
    "n_estimators": hp.quniform("n_estimators", 20, 700, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 1),
}

In [49]:
def RF_param_objective(params, train=True):
    # 超参数读取
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_RF = RandomForestRegressor(
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        random_state=12,
    )

    if train == True:
        res = -cross_val_score(
            reg_RF, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=15
        ).mean()
    else:
        res = reg_RF.fit(X_train, y_train)

    return res

In [50]:
def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )

In [51]:
RF_best_param = RF_param_search(50)

100%|██████████| 50/50 [07:10<00:00,  8.60s/trial, best loss: 0.2427352199984794] 


In [52]:
RF_best_param

{'max_features': 2, 'max_samples': 0.9297616244399035, 'n_estimators': 626.0}

In [53]:
RF_reg_B1 = RF_param_objective(RF_best_param, train=False)

In [54]:
mean_squared_error(RF_reg_B1.predict(X_train), y_train), mean_squared_error(
    RF_reg_B1.predict(X_test), y_test
)

(0.03702785158995616, 0.23246909513790037)

| 训练方式 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| 默认超参数 | <center>0.0351 |<center>0.2599 | <center>0.2508 |
| 小范围搜索 | <center>0.1943 |<center>0.2850 | <center>0.2760 |
| 大范围搜索（A） | <center>0.1328 |<center>0.2578 | <center>0.2500 |
| 高效搜索（B） | <center>0.0321 |<center>0.2414 | <center>0.2305 |

当然，此时由于诸多基础学习器的超参数未经过优化，因此在训练集上的过拟合风险加剧，但交叉验证的平均得分和测试集评分接近，说明交叉验证的平均得分仍然能够非常好的衡量模型泛化能力，说明模型搜索过程仍然有效。当然，通过和此前结果对比我们不难判断，训练集上的过拟合问题确实是由于超参数设置和搜索不完全导致的。

相比通过限制搜索空间来提高搜索效率，有的时候适度的放开模型的学习能力，哪怕让模型学到了一些局部规律，只要不严重影响测试集预测结果、不影响模型整体泛化能力，其实也没有太大问题，例如模型4。对于4号模型来说，由于不对基础模型的超参数进行限制，因此其学习能力是非常强的，但由于我们对其集成超参数进行了搜索和优化，因此仍然一定程度上对其学习能力进行了调整，在有限的调整空间下，我们无法获得类似模型3中获得的严谨结果，但仍然保证了其能够有效学习全局规律，因此最终预测集上结果有所提升。但由于还有很多不受控的超参数在影响模型的学习能力，因此模型在训练集的训练过程中也学习了非常多局部规律，导致其但从训练集上来看过拟合风险偏高。但就模型泛化能力、也就是对新数据的预测能力来说，模型4和模型3并无本质区别。

&emsp;&emsp;首先我们需要确定一点的是，从理论层面来说，超参数搜索最有效的方法一定是全参数大范围高精度搜索，如此得到的模型一定是能够最大程度学习全局规律的模型。但遗憾的是，在实际执行过程中，尤其是回归类问题，受限于当前算力条件，该方案几乎没有可执行性。因此需要考虑挑选出部分超参数进行搜索，而只要是挑选部分超参数出来进行搜索，其实就已经是不满足理论最优解的情况了，而具体要选哪些超参数出来搜索，也不存在理论最优解。因此，关于超参数的选取，其实也是需要多次尝试并且不断积累经验的。

&emsp;&emsp;总的来说，根据长期实践经验总结，集成相关超参数肯定需要进行搜索的，例如随机森林中的'max_features'、'n_estimators'和'max_samples'等，此外对于基础学习器的超参数来说，可以优先尝试max_depth、max_leaf_nodes这两个参数，该参数对决策树模型剪枝影响巨大。当然，是否要加入max_depth、max_leaf_nodes，甚至是其他基础分类器超参数进行搜索，可以通过少量次数迭代测试效果再决定。

In [55]:
RF_space = {
    "max_features": hp.choice("max_features", max_features_range),
    "n_estimators": hp.quniform("n_estimators", 200, 800, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 1),
    "max_depth": hp.quniform("max_depth", 20, 120, 1),
}


def RF_param_objective(params, train=True):
    # 超参数读取
    max_depth = int(params["max_depth"])
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_RF = RandomForestRegressor(
        max_depth=max_depth,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        random_state=12,
    )

    if train == True:
        res = -cross_val_score(
            reg_RF, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        ).mean()
    else:
        res = reg_RF.fit(X_train, y_train)

    return res


def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )


RF_param_search(50)

100%|██████████| 50/50 [07:52<00:00,  9.45s/trial, best loss: 0.2419435578184832] 


{'max_depth': 64.0,
 'max_features': 7,
 'max_samples': 0.996508030907065,
 'n_estimators': 423.0}

比不带入max_depth情况交叉验证平均得分更差，因此这里不宜带入max_depth进行搜索。接下来继续考虑带入max_leaf_nodes的情况：

In [56]:
RF_space = {
    "max_features": hp.choice("max_features", max_features_range),
    "n_estimators": hp.quniform("n_estimators", 200, 800, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 1),
    "max_depth": hp.quniform("max_depth", 20, 120, 1),
    "max_leaf_nodes": hp.quniform("max_leaf_nodes", 200, 600, 1),
}


def RF_param_objective(params, train=True):
    # 超参数读取
    max_depth = int(params["max_depth"])
    max_leaf_nodes = int(params["max_leaf_nodes"])
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_RF = RandomForestRegressor(
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        random_state=12,
    )

    if train == True:
        res = -cross_val_score(
            reg_RF, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        ).mean()
    else:
        res = reg_RF.fit(X_train, y_train)

    return res


def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )


RF_param_search(50)

100%|██████████| 50/50 [06:07<00:00,  7.34s/trial, best loss: 0.26178265241806653]


{'max_depth': 110.0,
 'max_features': 7,
 'max_leaf_nodes': 532.0,
 'max_samples': 0.9958345197920873,
 'n_estimators': 794.0}

&emsp;&emsp;当然，既然是诞生于实践过程的搜索策略，往往就会有长期实践过程中逐渐摸索出来的优化流程。对于这里我们看到的基于集成参数的高效搜索流程也不例外。整体来说，由于4号模型经过优化的超参数个数较少，因此模型在不同数据集上进行训练和预测时偏差较小但方差较大（例如max_depth取值为None时，就会根据不同数据集情况训练得到不同深度的基础树模型），目前来说，较为通用的围绕高效搜索流程算法特性设计的优化方法有以下两种，这两种方法的基本思路都源于交叉训练，希望通过类似交叉训练的过程来降低输出结果方差，提升结果稳定性，进而提高模型泛化能力：

方法一：在搜索得到一组最优超参数后，通过交叉训练的方式同时输出多组测试集的预测结果，然后求均值作为最终的预测结果；      
方法二：在超参数搜索过程中，不再使用验证集的平均得分作为搜索依据，换成验证集拼接而成的预测数据和标签之间的MSE作为搜索依据。

In [57]:
RF_reg_B1.set_params(n_jobs=-1)

RandomForestRegressor(max_features='log2', max_samples=0.9297616244399035,
                      n_estimators=626, n_jobs=-1, random_state=12)

方案一

In [58]:
RF_test_predict = 0
RF_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    RF_reg_B1.fit(X_train_part, y_train_part)
    RF_train_predict += RF_reg_B1.predict(X_train) / 5
    RF_test_predict += RF_reg_B1.predict(X_test) / 5

In [59]:
mean_squared_error(RF_train_predict, y_train)

0.06412037340179796

In [60]:
mean_squared_error(RF_test_predict, y_test)

0.23652047927469372

In [61]:
RF_reg_B1.fit(X_train, y_train)
mean_squared_error(RF_reg_B1.predict(X_train), y_train), mean_squared_error(
    RF_reg_B1.predict(X_test), y_test
)

(0.03702785158995616, 0.23246909513790034)

方案二

In [62]:
RF_space = {
    "max_features": hp.choice("max_features", max_features_range),
    "n_estimators": hp.quniform("n_estimators", 20, 700, 1),
    "max_samples": hp.uniform("max_samples", 0.2, 1),
}

In [63]:
def RF_param_objective(params, train=True):
    # 超参数读取
    n_estimators = int(params["n_estimators"])
    max_samples = params["max_samples"]

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_RF = RandomForestRegressor(
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        random_state=12,
        n_jobs=15,
    )

    oof_series = pd.Series(np.empty(X_train.shape[0]))

    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_RF.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_RF.predict(X_eval_part)

        res = mean_squared_error(oof_series, y_train)

    else:
        res = reg_RF.fit(X_train, y_train)

    return res

In [64]:
def RF_param_search(max_evals=500):
    return fmin(
        RF_param_objective, space=RF_space, algo=tpe.suggest, max_evals=max_evals
    )

In [65]:
RF_best_param = RF_param_search(50)

100%|██████████| 50/50 [03:06<00:00,  3.74s/trial, best loss: 0.2442098878390925] 


In [66]:
RF_best_param

{'max_features': 2, 'max_samples': 0.9999921130888294, 'n_estimators': 189.0}

In [67]:
RF_reg_B2 = RF_param_objective(RF_best_param, train=False)

In [68]:
mean_squared_error(RF_reg_B2.predict(X_train), y_train), mean_squared_error(
    RF_reg_B2.predict(X_test), y_test
)

(0.032821170137090024, 0.2327842107292657)

回归极端随机树超参数优化

In [69]:
ET_space = {
    "max_depth": hp.quniform("max_depth", 2, 50, 1),
    "n_estimators": hp.quniform("n_estimators", 20, 700, 1),
    "max_features": hp.choice("max_features", max_features_range),
}

In [70]:
def ET_param_objective(params, train=True):
    # 超参数读取
    max_depth = int(params["max_depth"])
    n_estimators = int(params["n_estimators"])

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_ET = ExtraTreesRegressor(
        max_depth=max_depth,
        n_estimators=n_estimators,
        max_features=max_features,
        random_state=12,
    )

    if train == True:
        res = -cross_val_score(
            reg_ET, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=15
        ).mean()
    else:
        res = reg_ET.fit(X_train, y_train)

    return res

In [71]:
def ET_param_search(max_evals=500):
    return fmin(
        ET_param_objective, space=ET_space, algo=tpe.suggest, max_evals=max_evals
    )

In [72]:
ET_best_param = ET_param_search(50)

100%|██████████| 50/50 [05:04<00:00,  6.09s/trial, best loss: 0.23461629604873618]


In [73]:
ET_best_param

{'max_depth': 41.0, 'max_features': 9, 'n_estimators': 584.0}

In [74]:
ET_reg = ET_param_objective(ET_best_param, train=False)
ET_reg

ExtraTreesRegressor(max_depth=41, max_features=0.6, n_estimators=584,
                    random_state=12)

In [75]:
mean_squared_error(ET_reg.predict(X_train), y_train),
mean_squared_error(ET_reg.predict(X_test), y_test)

0.22305638031008407

In [76]:
ET_reg.set_params(n_jobs=-1)

ExtraTreesRegressor(max_depth=41, max_features=0.6, n_estimators=584, n_jobs=-1,
                    random_state=12)

In [77]:
ET_test_predict = 0
ET_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    ET_reg.fit(X_train_part, y_train_part)
    ET_train_predict += ET_reg.predict(X_train) / 5
    ET_test_predict += ET_reg.predict(X_test) / 5

In [78]:
mean_squared_error(ET_train_predict, y_train),
mean_squared_error(ET_test_predict, y_test)

0.2252541561394263

In [79]:
def ET_param_objective(params, train=True):
    # 超参数读取
    max_depth = int(params["max_depth"])
    n_estimators = int(params["n_estimators"])

    if train == True:
        max_features = params["max_features"]

    else:
        max_features = max_features_range[params["max_features"]]

    # 模型创建
    reg_ET = ExtraTreesRegressor(
        max_depth=max_depth,
        n_estimators=n_estimators,
        max_features=max_features,
        random_state=12,
        n_jobs=-1,
    )

    oof_series = pd.Series(np.empty(X_train.shape[0]))

    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_ET.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_ET.predict(X_eval_part)

        res = mean_squared_error(oof_series, y_train)
    else:
        res = reg_ET.fit(X_train, y_train)

    return res

In [80]:
ET_best_param = ET_param_search(50)

100%|██████████| 50/50 [02:06<00:00,  2.53s/trial, best loss: 0.23324374995749944]


In [81]:
ET_reg2 = ET_param_objective(ET_best_param, train=False)

In [82]:
mean_squared_error(ET_reg2.predict(X_train), y_train),
mean_squared_error(ET_reg2.predict(X_test), y_test)

0.22320853438271698

回归GBDT超参数优化

In [83]:
GBR_space = {
    "n_estimators": hp.quniform("n_estimators", 20, 701, 1),
    "learning_rate": hp.uniform("learning_rate", 0.02, 0.2),
    "subsample": hp.uniform("subsample", 0.1, 1.0),
    "max_depth": hp.quniform("max_depth", 2, 20, 1),
}

In [84]:
def GBR_param_objective(params, train=True):
    n_estimators = int(params["n_estimators"])
    learning_rate = params["learning_rate"]
    subsample = params["subsample"]
    max_depth = int(params["max_depth"])

    reg_GBR = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        random_state=12,
    )
    if train == True:
        res = -cross_val_score(
            reg_GBR, X_train, y_train, scoring="neg_mean_squared_error", n_jobs=-1
        ).mean()
    else:
        res = reg_GBR.fit(X_train, y_train)
    return res

In [85]:
def GBR_param_search(max_evals=500):
    return fmin(
        GBR_param_objective, space=GBR_space, algo=tpe.suggest, max_evals=max_evals
    )

In [86]:
GBR_best_param = GBR_param_search(50)

100%|██████████| 50/50 [09:29<00:00, 11.40s/trial, best loss: 0.20933920422665336]


In [87]:
GBR_best_param

{'learning_rate': 0.02518721022024758,
 'max_depth': 8.0,
 'n_estimators': 550.0,
 'subsample': 0.7943547875565606}

In [88]:
GBR_reg = GBR_param_objective(GBR_best_param, train=False)

In [89]:
GBR_reg

GradientBoostingRegressor(learning_rate=0.02518721022024758, max_depth=8,
                          n_estimators=550, random_state=12,
                          subsample=0.7943547875565606)

In [90]:
mean_squared_error(GBR_reg.predict(X_train), y_train),
mean_squared_error(GBR_reg.predict(X_test), y_test)

0.20238422969052877

In [91]:
GBR_test_predict = 0
GBR_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    GBR_reg.fit(X_train_part, y_train_part)
    GBR_train_predict += GBR_reg.predict(X_train) / 5
    GBR_test_predict += GBR_reg.predict(X_test) / 5

In [92]:
mean_squared_error(GBR_train_predict, y_train),
mean_squared_error(GBR_test_predict, y_test)

0.19973202850449978

In [93]:
def GBR_param_objective(params, train=True):
    n_estimators = int(params["n_estimators"])
    learning_rate = params["learning_rate"]
    subsample = params["subsample"]
    max_depth = int(params["max_depth"])

    reg_GBR = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        random_state=12,
    )

    oof_series = pd.Series(np.empty(X_train.shape[0]))

    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_GBR.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_GBR.predict(X_eval_part)

        res = mean_squared_error(oof_series, y_train)
    else:
        res = reg_GBR.fit(X_train, y_train)
    return res

In [94]:
GBR_best_param = GBR_param_search(50)

100%|██████████| 50/50 [49:30<00:00, 59.42s/trial, best loss: 0.21198462655059314]  


In [95]:
GBR_best_param

{'learning_rate': 0.07858979613062272,
 'max_depth': 8.0,
 'n_estimators': 285.0,
 'subsample': 0.8397991762994271}

In [96]:
GBR_reg = GBR_param_objective(GBR_best_param, train=False)
GBR_reg

GradientBoostingRegressor(learning_rate=0.07858979613062272, max_depth=8,
                          n_estimators=285, random_state=12,
                          subsample=0.8397991762994271)

In [97]:
mean_squared_error(GBR_reg.predict(X_train), y_train), 
mean_squared_error(GBR_reg.predict(X_test), y_test)

0.2034893329166238

In [98]:
GBR_test_predict = 0
GBR_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    GBR_reg.fit(X_train_part, y_train_part)
    GBR_train_predict += GBR_reg.predict(X_train) / 5
    GBR_test_predict += GBR_reg.predict(X_test) / 5

In [99]:
mean_squared_error(GBR_train_predict, y_train),
mean_squared_error(GBR_test_predict, y_test)

0.19705372635366933

In [100]:
def GBR_param_objective(params, train=True):
    n_estimators = int(params["n_estimators"])
    learning_rate = params["learning_rate"]
    subsample = params["subsample"]
    max_depth = int(params["max_depth"])

    reg_GBR = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        random_state=12,
    )

    oof_series = pd.Series(np.empty(X_train.shape[0]))

    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_GBR.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_GBR.predict(X_eval_part)

        res = mean_squared_error(oof_series, y_train)
    else:
        res = reg_GBR.fit(X_train, y_train)
    return res

In [101]:
GBR_best_param = GBR_param_search(50)

100%|██████████| 50/50 [45:03<00:00, 54.08s/trial, best loss: 0.21141578335249117] 


In [102]:
GBR_best_param

{'learning_rate': 0.06061896900844315,
 'max_depth': 8.0,
 'n_estimators': 544.0,
 'subsample': 0.5539249758481197}

In [103]:
GBR_reg2 = GBR_param_objective(GBR_best_param, train=False)

In [104]:
GBR_reg2

GradientBoostingRegressor(learning_rate=0.06061896900844315, max_depth=8,
                          n_estimators=544, random_state=12,
                          subsample=0.5539249758481197)

In [105]:
mean_squared_error(GBR_reg2.predict(X_train), y_train),
mean_squared_error(GBR_reg2.predict(X_test), y_test)

0.2048150038905001

| 模型 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| <center>随机森林 | <center>0.0351 |<center> 0.2599 | <center>0.2508 |
| <center>随机森林_OPT | <center>0.0321 |<center>0.2414 | <center>0.2305 |
| <center>极端随机树 | <center>0 |<center> 0.2581 | <center>0.2436 |
| <center>极端随机树_OPT | <center>1.36e-07 |<center> 0.2346 | <center>0.2257 |
| <center>GBDT | <center>0.2605 |<center> 0.2870 | <center>0.2777 |
| <center>GBDT_OPT | <center>0.03545 |<center> 0.2091 | <center>0.1916 |

In [106]:
dump(RF_reg, "./model/RF_reg_B1.joblib")
dump(ET_reg, "./model/ET_reg.joblib")
dump(GBR_reg, "./model/GBR_reg.joblib")

['./model/GBR_reg.joblib']